## Objective 2: On-Time vs Late Delivery Analysis

### Objective
The purpose of this analysis is to evaluate whether delivered orders arrive on time compared to the estimated delivery dates provided to customers.

Unlike the previous objective, which focused on overall delivery duration, this analysis shifts the perspective to **expectation versus reality**. The goal is to understand how often deliveries meet, exceed, or miss the promised delivery timeline.

---

### Scope
This analysis focuses exclusively on orders with a `delivered` status.

Only orders that contain both:
- `order_delivered_customer_date`
- `order_estimated_delivery_date`

are considered eligible for analysis. Orders with missing or logically inconsistent delivery dates are separated and reported as data quality findings, rather than being silently excluded.

---

### Metric Definition
Delivery performance is evaluated by comparing the actual delivery date (`order_delivered_customer_date`) against the estimated delivery date (`order_estimated_delivery_date`).

Based on this comparison, each order can be categorized as:
- Delivered earlier than estimated
- Delivered on the estimated date
- Delivered later than estimated

Additional metrics may include the number of days early or late, depending on the final definition of on-time delivery.

---

### Approach
The analysis prioritizes clarity and consistency over complex assumptions. Definitions such as “on-time” and “late” are established explicitly and applied uniformly across the dataset.

The focus is on describing delivery performance patterns observed in the data, rather than evaluating performance against external benchmarks or service-level agreements.

---

### Interpretation Notes
- All interpretations are relative to the estimated delivery dates provided in the dataset.
- No external expectations or business targets are assumed.
- Results are intended to provide context for further breakdowns (e.g., by location or seller) in subsequent analyses.


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [2]:
IS_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ

DATA_DIR = (
    "/kaggle/input/brazilian-ecommerce"
    if IS_KAGGLE
    else "../data/raw"
)

OUTPUT_DIR = (
    "/kaggle/working"
    if IS_KAGGLE
    else "../data/reports"
)

In [ ]:
def load_dataset(path):
    '''
    Loads a CSV file into Pandas Dataframe.
    
    args:
        path(string): File path to the csv from this file
    
    return:
        pd.DataFrame: Loaded data
    '''
    return pd.read_csv(path)

def export_to_csv(dataFrame, fileName):
    dataFrame.to_csv(f"{OUTPUT_DIR}/{fileName}")

In [4]:
# load the orders dataset
orders_path = f'{DATA_DIR}/olist_orders_dataset.csv'
orders = load_dataset(orders_path)
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


### Data Cleaning

In [ ]:
# Data Cleaning

delivered_orders = orders[orders['order_status'] == "delivered"].copy()
delivered_orders

# get the missing/invalids for reports
## get the order from column `order_delivered_customer_date` with value NaN for reports
nan_delivered_orders = delivered_orders[delivered_orders['order_delivered_customer_date'].isna() | delivered_orders['order_estimated_delivery_date'].isna()]

export_to_csv(nan_delivered_orders, "estimated_or_delivered_orders_nan.csv")

# removes the missing dates
cleaned_df_orders = delivered_orders.dropna(subset=['order_delivered_customer_date', 'order_estimated_delivery_date']).copy()

# check for invalids dates in column estimated_or_delivered_orders_nan & order_delivered_customer_date
# check for the datetime format of the column estimated_or_delivered_orders_nan & order_delivered_customer_date
cleaned_df_orders['order_delivered_customer_date'] = pd.to_datetime(cleaned_df_orders['order_delivered_customer_date'], errors='coerce')
cleaned_df_orders['order_estimated_delivery_date'] = pd.to_datetime(cleaned_df_orders['order_estimated_delivery_date'], errors='coerce')

invalids_estimate_delivered_orders = cleaned_df_orders[cleaned_df_orders['order_delivered_customer_date'].isna() | cleaned_df_orders['order_estimated_delivery_date'].isna()]
export_to_csv(invalids_estimate_delivered_orders, "invalids_estimate_delivered_orders.csv")

# removes the invalid dates
cleaned_df_orders = cleaned_df_orders.dropna(subset=['order_delivered_customer_date', 'order_estimated_delivery_date'])

In [ ]:
# create new column `delivery_delay_days`
cleaned_df_orders['delivery_delay_days'] = (cleaned_df_orders['order_delivered_customer_date'] - cleaned_df_orders['order_estimated_delivery_date']).dt.days

# do sanity checks
median_delivery_days = cleaned_df_orders['delivery_delay_days'].median()
min_delivery_days = cleaned_df_orders['delivery_delay_days'].min()
max_delivery_days = cleaned_df_orders['delivery_delay_days'].max()

print("negative number means earlier than estimated date, while positive means after estimation dates")
print(f"median = {median_delivery_days} days, min = {min_delivery_days} days, max = {max_delivery_days} days")

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,-8
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,-6
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,-18
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,-13
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,-10
...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,-11
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,-2
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,-6
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,-21


#### Conclusion:

The estimated delivery dates appear to be conservatively set, which leads to a majority of orders being delivered earlier than the promised date. 

This conservative estimation strategy may help reduce the risk of late deliveries from a customer expectation perspective.

In [ ]:
# now let's do the classification
cleaned_df_orders['delivery_category'] = 'late'
cleaned_df_orders.loc[
    cleaned_df_orders['delivery_delay_days'] < 0,
    'delivery_category'
] = 'early'
cleaned_df_orders.loc[
    cleaned_df_orders['delivery_delay_days'].between(0,1),
    'delivery_category'
] = 'on-time'


# calculate each category
category_counts = cleaned_df_orders['delivery_category'].value_counts()
category_percentage = cleaned_df_orders['delivery_category'].value_counts(normalize=True) * 100

# sanity check. the total should be 100
# category_percentage.sum()

# create dataframe for the summary reports
delivery_category_summary = pd.DataFrame({
    'count': category_counts,
    'percentage': category_percentage
}).reset_index()

delivery_category_summary.columns = [
    'delivery_category', 'count', 'percentage'
]

export_to_csv(delivery_category_summary, "delivery_category_summary.csv")

### Conclusion:

I analyzed the dataset to compare actual delivery dates with the estimated delivery dates. The results show that most orders are delivered earlier than the estimated delivery dates.

This does not necessarily indicate good delivery performance, as the estimated delivery dates may be conservative and tend to overestimate actual delivery time.

